![GridSearch](slides/7_29.png)

![GridSearch](slides/7_30.png)

In [1]:
import pandas as pd
import seaborn as sns

# Iris データセット
iris = sns.load_dataset('iris')

# species が setosa のデータを除去
df = iris.query('species!="setosa"')
# speciesを 0, 1 にするためにダミー変数化
df = pd.get_dummies(data=df, drop_first=True)

# データセットの確認
df.iloc[[0, 50]]

# 説明変数、目的変数を作成
X = df[['petal_length', 'petal_width']]
Y = df.species_virginica

### グリッドサーチによるパラメータのチューニング

GridSearchCV() にモデルとパラメータを渡し fit() を実行するだけ。

- cv=10: 10分割で実行
- scoring='roc_auc': 精度はAUC

指定できる精度指標は以下を参照<br>
https://scikit-learn.org/stable/modules/model_evaluation.html

In [2]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV

# SVM Classifier を作成
model = SVC()

# 探索パラメータの設定例 (1)
#param_range = [0.0001, 0.001, 0.01, 0.1, 1.0, 10.0, 100.0, 1000.0]
#param_degree = list(range(3, 25, 3))

# 探索パラメータの設定例 (2)
# - 実行時間短縮のためパラメータも簡略化
param_range = [0.01, 0.1, 1.0, 10.0]
param_degree = list(range(3, 10, 3))

# グリッドの設定
grid = [{'kernel': ['linear'], 'C': param_range},
        {'kernel': ['rbf'], 'C': param_range, 'gamma': param_range},
        {'kernel': ['poly'], 'C': param_range, 'degree': param_degree}]

# グリッドサーチの作成、実行 # n_jobs=-1は計算資源をできるだけ多く使う
gs = GridSearchCV(model, param_grid=grid, scoring='roc_auc', cv=10, n_jobs=-1)
gs.fit(X, Y)

# 結果の表示
print('best AUC={:.3f}'.format(gs.best_score_))
print('parameter:', gs.best_params_)

best AUC=0.996
parameter: {'C': 10.0, 'gamma': 1.0, 'kernel': 'rbf'}


グリッドサーチの実行結果は表 cv_results_ にまとめられている。<br>
pandas DataFrame に変換し、AUCが上位のパラメータを取得することもできる。

In [3]:
df = pd.DataFrame(gs.cv_results_).sort_values('mean_test_score', ascending=False)
df[['mean_test_score', 'params']].head(5)

,mean_test_score,params
27,0.996,"{'C': 1.0, 'degree': 6, 'kernel': 'poly'}"
22,0.996,"{'C': 0.01, 'degree': 9, 'kernel': 'poly'}"
18,0.996,"{'C': 10.0, 'gamma': 1.0, 'kernel': 'rbf'}"
31,0.992,"{'C': 10.0, 'degree': 9, 'kernel': 'poly'}"
28,0.992,"{'C': 1.0, 'degree': 9, 'kernel': 'poly'}"


### GridSearchCV を用いた交差検証法の実行

- 最良のパラメータを知りたいだけならば GridSearchCV を用いて交差検証法の実行可能

In [4]:
from sklearn.tree import DecisionTreeClassifier

# 決定木を作成
model = DecisionTreeClassifier()

# 探索パラメータ、グリッドの設定
param_range = list(range(1, 16, 1))
grid = [{'max_depth': param_range}]

# グリッドサーチの作成、実行
gs = GridSearchCV(model, param_grid=grid, scoring='roc_auc', cv=10, n_jobs=-1)
gs.fit(X, Y)

# 結果の表示
print('best AUC={:.3f}'.format(gs.best_score_))
print('parameter:', gs.best_params_)

best AUC=0.952
parameter: {'max_depth': 2}


In [5]:
df = pd.DataFrame(gs.cv_results_).sort_values('mean_test_score', ascending=False)
df[['mean_test_score', 'params']].head(5)

,mean_test_score,params
1,0.952,{'max_depth': 2}
2,0.938,{'max_depth': 3}
3,0.938,{'max_depth': 4}
4,0.938,{'max_depth': 5}
5,0.938,{'max_depth': 6}
